In [1]:
import os
import json
import openai
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv
load_dotenv('/home/pervinco/LLM-tutorials/keys.env')
openai_api_key = os.getenv('GRAVY_LAB_OPENAI')

from src.data_processor import translate_and_convert_to_string, process_vision_result, extract_workstyle_info

In [ ]:
n_iter = 10
temperature = 0.1
gt_path = "./data/amy_gt.json"
data_path = "./data/amy_culture_fit.json"
embed_model = "text-embedding-3-large"

output_dir = "./result"
csv_file_name = f"CoT_N-{n_iter}.csv"

In [3]:
client = openai.OpenAI(api_key=openai_api_key)

In [4]:
with open(data_path, 'r', encoding='utf-8') as file:
    hr_data_dict = json.load(file)

with open(data_path, 'r', encoding="utf-8") as file:
    gt_data_dict = json.load(file)

In [5]:
processed_data_summary = translate_and_convert_to_string(hr_data_dict['summaryResult'])
vision_data = process_vision_result(hr_data_dict['visionResult'], hr_data_dict['summaryResult'])
workstyle_data = extract_workstyle_info(hr_data_dict['workstyleResult'], hr_data_dict['summaryResult'])

In [6]:
print(processed_data_summary)


채용 권장 수준, 입사 후 적응 기간, 조기 퇴사 가능성 : 
조기 퇴사 가능성: 낮음
입사 후 적응 기간: 보통
채용 권장 수준: 보통

검사 항목별 결과 : 
5) 사고방식이 기업 비전,가치관에 부합하는가?: 매우 그렇다
2) 타 팀, 타 구성원과의 원만한 협업을 기대할 수 있는가?: 그렇다
3) 경영진, 상급자와의 원활한 소통을 기대할 수 있는가?: 그렇다
4) 기업이 추구하는 일하는 방식과 부합하는가?: 매우 그렇다
1) 구성원들과 원활한 소통이 가능한가?: 그렇다

이직 스트레스 요인 : 
공정인사

위험 성향 : 오만형


In [7]:
print(vision_data)

{'company_top_keywords': '전문성:4.6, 성과:4.2, 사회공헌:4.2', 'company_remaining_keywords': '상생:4.0, 최고지향:3.6, 고객:3.0, 성장:3.0', 'compute_top_keywords': '창조:5.0, 열정:5.0, 혁신:4.58', 'compute_remaining_keywords': '신속성:4.23, 사회공헌:4.17, 고객:4.0, 성장:3.85, 소통:3.75, 성과:3.33, 도전:3.21, 상생:3.13, 최고지향:2.78, 문제해결:2.5, 인재:2.27, 즐거움:1.5, 전문성:0.38', 'compute_vision_total_evalation': '보통'}


In [8]:
print(workstyle_data)

for k, v in workstyle_data.items():
    print(k, v)

{'company_keywords': '스피드형:3.2, 책임형:2.7, 목표지향형:4.0, 끈기형:3.0, 긍정형:3.5, 유니크형:2.9, 혁신형:3.4, 도전형:3.0, 스마트형:3.5, 윤리형:2.2, 성취형:2.2, 열린사고형:3.3, 솔선수범형:3.5, 몰입형:4.3, 신뢰형:1.8, 자기확신형:2.7', 'compute_keywords': '스피드형:4.33, 책임형:0.63, 목표지향형:3.33, 끈기형:0.67, 긍정형:2.5, 유니크형:5.0, 혁신형:5.0, 도전형:3.5, 스마트형:2.75, 윤리형:1.25, 성취형:3.44, 열린사고형:3.75, 솔선수범형:1.88, 몰입형:3.44, 신뢰형:0.65, 자기확신형:3.0', 'workstyle_match_percentage': 18.75, 'workstyle_company_total_score': 49.2, 'workstyle_compute_total_score': 45.12, 'comparison_ratio': 91.70731707317073, 'compute_workstyle_total_evalation': '우수'}
company_keywords 스피드형:3.2, 책임형:2.7, 목표지향형:4.0, 끈기형:3.0, 긍정형:3.5, 유니크형:2.9, 혁신형:3.4, 도전형:3.0, 스마트형:3.5, 윤리형:2.2, 성취형:2.2, 열린사고형:3.3, 솔선수범형:3.5, 몰입형:4.3, 신뢰형:1.8, 자기확신형:2.7
compute_keywords 스피드형:4.33, 책임형:0.63, 목표지향형:3.33, 끈기형:0.67, 긍정형:2.5, 유니크형:5.0, 혁신형:5.0, 도전형:3.5, 스마트형:2.75, 윤리형:1.25, 성취형:3.44, 열린사고형:3.75, 솔선수범형:1.88, 몰입형:3.44, 신뢰형:0.65, 자기확신형:3.0
workstyle_match_percentage 18.75
workstyle_company_total_score 49.2
workstyle_compu

In [9]:
vision_prompt = """
주어진 데이터는 기업 비전과 피검사자의 비전 점수를 기반으로 분석한 결과입니다. 다음은 분석 단계입니다:

1. **기업의 비전 상위 3개 선별**:
   기업의 비전에서 점수가 높은 상위 3개의 키워드를 선택합니다. 예를 들어, ["전문성", "사회공헌", "성과"]와 같이 구성됩니다.

2. **피검사자의 비전 상위 3개 선별**:
   피검사자의 비전에서 점수가 높은 상위 3개의 키워드를 선택합니다. 예를 들어, ["창조", "열정", "혁신"]와 같이 구성됩니다.

3. **동일 키워드 점수 차이 분석**:
   기업과 피검사자 상위 3개 키워드 중 동일한 키워드가 있을 경우, 두 점수의 차이를 계산합니다. 이를 "equal_top3"라 부릅니다. 차이가 큰 경우 우려를 표현하며, 차이가 작은 경우 긍정적인 평가를 제공합니다. 예를 들어:
   - "전문성" (기업: 4.6, 피검사자: 0.38) → 큰 차이로 인해 우려 사항.
   - "사회공헌" (기업: 4.2, 피검사자: 4.17) → 매우 유사함으로 긍정 평가.

4. **일치하지 않는 키워드 분석**:
   기업 상위 3개 키워드와 일치하지 않는 피검사자의 상위 키워드를 "noeq_top3"로 구분합니다. 이를 통해 피검사자가 독자적으로 높은 관심을 가진 비전을 파악합니다. 예를 들어, "창조", "열정".

5. **결과 서술**:
   - 기업이 중요시하는 비전과 피검사자가 중요시하는 비전을 각각 서술합니다.
   - "equal_top3"에서 차이가 1.8 이상인 키워드에 대해 우려를 표현하되, 부정적인 내용은 피합니다.
   - "equal_top3"에서 차이가 0.5 이하인 경우 긍정적으로 표현합니다.
   - 피검사자의 점수가 기업보다 높은 경우, 강한 긍정과 추가적 가치 창출 가능성을 언급합니다.

6. **최종 결과 문장 생성**:
   300자 이내의 문장으로 피검사자가 기업 비전과 어느 정도 정렬되어 있는지를 서술합니다. 
   반드시 존댓말을 사용하며, 피검사자가 기업에 적합한지 겸손한 어조로 표현합니다. 
   문장의 끝부분에는 "전반적으로 피검사자와 기업간 비전 fit은 [높음, 보통, 낮은] 편입니다." 로 마무리 지어 주세요.
   또한 아래 예시를 참고하여 작성합니다:

   "기업은 전문성, 성과, 사회공헌을 중요시하며, 피검사자는 창조, 열정, 혁신에 높은 가치를 두고 있습니다. 피검사자는 사회공헌에 대한 관심이 기업과 유사하며, 이는 긍정적으로 평가됩니다. 그러나 전문성에 대한 피검사자의 관심이 상대적으로 낮아 우려가 있을 수 있습니다. 반면, 피검사자는 성과에 대한 관심이 기업보다 낮지만, 창조와 열정에서 높은 점수를 보이며 기업에 추가적인 가치를 제공할 가능성이 있습니다. 전반적으로 피검사자와 기업간 비전 fit은 보통입니다."
"""

workstyle_prompt = """
목표: 기업이 중요하게 생각하는 업무 스타일을 기준으로 피검사자의 업무 스타일이 얼마나 일치하는지를 평가하는 문장을 작성하는 것입니다. 이 문장은 피검사자가 기업의 업무 스타일에 적합한지를 평가하여 조직 적응 가능성을 판단하는 데 사용됩니다.

작업 설명: 다음 과정을 따라 피검사자가 기업의 업무 스타일에 얼마나 잘 맞는지를 평가하는 문장을 작성하세요.

과정:
1. 기업의 업무 성향 키워드 점수와 피검사자의 업무 성향 키워드 점수 간 차이를 계산합니다. 차이를 계산한 결과를 `ws_score_diff`라는 딕셔너리에 저장합니다. (`ws_score_diff = (키워드: company 점수 - compute 점수)` 형식)
2. `ws_score_diff`에서 점수 차이가 0.5 이하인 키워드에 대해 다음 문장을 생성합니다:
   - 점수 차이가 0 이상이면: "피검사자는 (점수 차이가 0.5 이하인 키워드들)에서 기업의 기대치를 충족하여 긍정적 평가를 받을 수 있습니다."
   - 점수 차이가 음수이면: "피검사자는 점수 차이가 음수인 키워드들에서 기업의 기대치를 초과하여 매우 긍정적 평가를 받을 수 있습니다."
   - 두 개의 결과를 종합해서 하나의 문장을 생성합니다.
3. `ws_score_diff`에서 점수 차이가 1.8 이상인 키워드에 대해 다음 문장을 생성합니다:
   - "(점수차가 1.8이상인 키워드들)에 대해 기업의 기대치와 차이가 있습니다."
4. `compute_workstyle_total_evaluation` 값(“우수”, “보통”, “검토필요”)에 따라 전반적인 fit 평가를 추가합니다:
   - "우수": "전반적으로 피검사자와 기업간 업무 성향 fit은 높은 편입니다."
   - "보통": "전반적으로 피검사자와 기업간 업무 성향 fit은 보통입니다."
   - "검토필요": "전반적으로 피검사자와 기업간 업무 성향 fit은 낮은 편입니다."
5. 과정 2, 3, 4에서 생성된 문장을 종합하여 300자 이내로 작성합니다. 

중요 사항:
- 문장은 한글로 작성하며, 항상 존댓말을 사용하세요.
- 피검사자 점수가 기업 점수보다 높을 경우 이를 긍정적으로 표현하세요. ("약간의 차이" 또는 부정적인 뉘앙스는 피해주세요.)
- 각 키워드는 제공된 명칭 그대로 사용하고, 점수를 문장에 포함하지 마세요.
- 동일한 문장을 반복하지 않고 간결하고 명확하게 작성하세요.
- 최종 문장의 글자 수는 300자를 초과하지 않도록 주의하세요.

최종 출력: 피검사자가 기업의 업무 스타일에 얼마나 잘 맞는지 평가한 한 문장을 반환합니다. 이 문장은 피검사자의 적합성과 차이를 설명하며 fit 평가를 포함합니다.
"""

summary_prompt = """
목표: 채용 피검사자의 결과를 바탕으로, 피검사자가 해당 기업에 얼마나 적합한지 종합적으로 평가하고, 채용 여부를 결정하는 데 필요한 코멘트를 작성하는 것입니다.

작업 설명: 피검사자의 평가 데이터를 분석하고, 단계별로 정보를 종합하여 최종적으로 200자 이내의 코멘트를 작성합니다. 모든 문장은 한글로 작성하고, 존댓말과 겸손한 어체를 사용하며, 띄어쓰기를 포함하여 200자 이내로 제한해야 합니다.

주의 사항: 단계별로 결과를 도출하지만, 반환하는 값은 5단계에서 생성한 최종 코멘트만 반환해야합니다. 5단계를 제외한 다른 단계의 결과를 반환하지 않도록 주의하세요.

---

**1단계:**  
- "recruitentQuestions" 값에서 ["그렇다", "매우 그렇다"]에 해당하는 항목들을 기반으로 피검사자가 가진 긍정적인 점을 서술하세요.  
- 예: "피검사자는 팀 내 협력과 소통에서 긍정적인 평가를 받았습니다."

**2단계:**  
- "fued" 값이 피검사자의 갈등 유발 요인을 나타냅니다.  
- 갈등 유발 요인이 있을 경우, 조직 내에서 발생할 수 있는 문제와 갈등을 강하게 경고하는 문장을 작성하세요.  
- "fued" 값이 "없음"일 경우, 해당 단계는 생략하고 다음 단계로 넘어갑니다.  
- 예: "투쟁형 성향으로 인해 과도한 경쟁과 갈등이 우려되며, 팀 분위기에 부정적인 영향을 미칠 수 있습니다."

**3단계:**  
- "turnOverFactors" 값이 "없음"이 아닐 경우, "OO에 대한 스트레스 요인이 있으므로,"로 시작하여 기업이 대응할 수 있는 방법을 제시하세요.  
- 확정적인 어체를 피하고, "필요할 수 있습니다"와 같은 제시형 어체를 사용하세요.  
- "turnOverFactors" 값이 "없음"일 경우, 해당 단계는 생략하고 다음 단계로 넘어갑니다.  
- 예: "공정인사에 대한 스트레스 요인이 있으므로, 인사정책에 대한 명확한 안내가 필요할 수 있습니다."

**4단계:**  
- "additionalInformation" 값 중 "입사 후 적응 기간"과 "조기 퇴사 가능성"을 활용해 간략히 서술하세요.  
- 예: "입사 후 적응 기간은 보통이며, 조기 퇴사 가능성은 낮습니다."

**5단계:**  
- 1, 2, 3, 4단계에서 도출한 결과를 종합하여, 200자 이내로 최종 코멘트를 작성하세요.  
- 존댓말과 겸손한 어체를 사용하며, "채용을 권장합니다" 또는 "채용을 권장하지 않습니다"와 같은 끝맺음 문장은 사용하지 않습니다.  
- 예: "피검사자는 기업의 비전과 가치관에 잘 부합하며, 타 팀 및 구성원과의 협업과 소통에서도 긍정적인 평가를 받았습니다. 그러나 오만형 성향이 감지되어 팀원 간 신뢰를 저해하고 적대적인 분위기를 형성할 수 있습니다. 이는 조직 내 갈등을 유발할 수 있으므로 주의가 필요합니다. 공정인사에 대한 스트레스 요인이 있으므로, 인사 정책에 대한 명확한 안내가 필요할 수 있습니다. 입사 후 적응 기간은 보통이며, 조기 퇴사 가능성은 낮습니다."
"""

In [10]:
vision_input = f"""
기업의 비전 데이터: {hr_data_dict["visionResult"]['company']}
피검사자의 비전 데이터: {hr_data_dict["visionResult"]['compute']}
"""

workstyle_input = f"""
company_keywords : {workstyle_data['company_keywords']}
compute_keywords : {workstyle_data['compute_keywords']}
workstyle_company_total_score : {workstyle_data['workstyle_company_total_score']}
workstyle_compute_total_score : {workstyle_data['workstyle_compute_total_score']}
workstyle_compute_total_score : {workstyle_data['workstyle_match_percentage']}
comparison_ratio: {workstyle_data['comparison_ratio']}
compute_workstyle_total_evaluation : {workstyle_data['compute_workstyle_total_evalation']}
"""

summary_input = f"""
additionalInformation : {hr_data_dict['summaryResult']['additionalInformation']}
recruitentQuestions : {hr_data_dict['summaryResult']['recruitentQuestions']}
turnOVerFactors : {hr_data_dict['summaryResult']['turnOverFactors']}
fued :{hr_data_dict['summaryResult']['fued']}
"""

In [11]:
def run_openai_api(n_iter, prompt, input, temperature):
    results = []
    for i in range(n_iter):
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": input}
            ],
            temperature=temperature
        )
        response_content = completion.choices[0].message.content
        results.append({
            "iteration": i + 1,
            "response": response_content
        })
    return results

In [12]:
print(f"company top3 : {vision_data['company_top_keywords']}\n")
print(f"compute top3 : {vision_data['compute_top_keywords']}\n")

print(f"compnay remain : {vision_data['company_remaining_keywords']}\n")
print(f"compute remain : {vision_data['compute_remaining_keywords']}\n")

print(f"fianl eval : {vision_data['compute_vision_total_evalation']}")

company top3 : 전문성:4.6, 성과:4.2, 사회공헌:4.2

compute top3 : 창조:5.0, 열정:5.0, 혁신:4.58

compnay remain : 상생:4.0, 최고지향:3.6, 고객:3.0, 성장:3.0

compute remain : 신속성:4.23, 사회공헌:4.17, 고객:4.0, 성장:3.85, 소통:3.75, 성과:3.33, 도전:3.21, 상생:3.13, 최고지향:2.78, 문제해결:2.5, 인재:2.27, 즐거움:1.5, 전문성:0.38

fianl eval : 보통


In [13]:
vision_results = run_openai_api(n_iter, vision_prompt, vision_input, temperature)
workstyle_results = run_openai_api(n_iter, workstyle_prompt, workstyle_input, temperature)
summary_results = run_openai_api(n_iter, summary_prompt, summary_input, temperature)

In [14]:
def create_results_dataframe(vision_results, workstyle_results):
    """Create separate dataframes for vision and workstyle results"""
    vision_df = pd.DataFrame(vision_results)
    vision_df['type'] = 'vision'

    workstyle_df = pd.DataFrame(workstyle_results)
    workstyle_df['type'] = 'workstyle'

    summary_df = pd.DataFrame(summary_results)
    summary_df['type'] = 'summary'
    
    # Combine the dataframes
    combined_df = pd.concat([vision_df, workstyle_df, summary_df], ignore_index=True)
    return combined_df

In [15]:
df = create_results_dataframe(vision_results, workstyle_results)
df.to_csv(os.path.join(output_dir, csv_file_name), index=False, encoding='utf-8-sig')
print(f"Results saved to {os.path.join(output_dir, csv_file_name)}")

Results saved to ./result/CoT_N-1.csv


In [16]:
df = pd.read_csv(f"{output_dir}/{csv_file_name}")
responses = df["response"].tolist()

In [17]:
def calculate_embedding_similarity_and_embeddings(responses, client):
    embeddings = []
    for response in responses:
        embedding_response = client.embeddings.create(
            input=response,
            model=embed_model
        )
        embeddings.append(embedding_response.data[0].embedding)

    embeddings = np.array(embeddings)
    similarity_matrix = cosine_similarity(embeddings)
    mean_similarity = similarity_matrix.mean()
    
    return mean_similarity, embeddings

In [18]:
def calculate_lexical_similarity(responses):
    """Calculate lexical overlap between responses"""
    def lexical_overlap(response1, response2):
        words1 = set(response1.split())
        words2 = set(response2.split())
        return len(words1 & words2) / len(words1 | words2)
    
    lexical_similarities = [
        lexical_overlap(responses[i], responses[j])
        for i in range(len(responses)) for j in range(i + 1, len(responses))
    ]
    mean_lexical_similarity = sum(lexical_similarities) / len(lexical_similarities)
    return mean_lexical_similarity

In [19]:
def analyze_responses(df, client):
    """Analyze responses for each type (vision and workstyle)"""
    results = {}
    
    for response_type in ['vision', 'workstyle', 'summary']:
        type_responses = df[df['type'] == response_type]['response'].tolist()
        
        # Calculate semantic similarity
        mean_similarity, embeddings = calculate_embedding_similarity_and_embeddings(type_responses, client)
        
        # Calculate lexical similarity
        mean_lexical_similarity = calculate_lexical_similarity(type_responses)
        
        results[response_type] = {
            'semantic_similarity': mean_similarity,
            'lexical_similarity': mean_lexical_similarity,
            'embeddings': embeddings
        }
    
    return results

In [20]:
# Analyze responses
analysis_results = analyze_responses(df, client)

# Print results for both types
for response_type, metrics in analysis_results.items():
    print(f"\nResults for {response_type.upper()}:")
    print(f"Semantic Similarity (mean): {metrics['semantic_similarity']:.2f}")
    print(f"Lexical Overlap (mean): {metrics['lexical_similarity']:.2f}")

ZeroDivisionError: division by zero